In [1]:
from API_call import API_call
from gamePks import *
import statsapi as mlb
import pandas as pd

from tables import *
import relationships

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [2]:
db.db_engine.table_names()

['actions',
 'credits',
 'game_batting_stats',
 'game_player_links',
 'game_team_links',
 'games',
 'hit_data',
 'matchups',
 'pitch_data',
 'pitches',
 'players',
 'plays',
 'runners',
 'team_records',
 'teams',
 'venues']

In [3]:
# Yankees games on my birthday
stmt= """
SELECT 
    pk
FROM
    games
WHERE
    pk 
IN
    (
    SELECT
        gamePk
    FROM
        game_team_links
    WHERE
        team_id 
    IN
        (SELECT
            id
        FROM 
            teams
        WHERE
            name = 'New York Yankees')
    )
AND
    strftime('%m-%d',dateTime)='06-09'
    """

In [4]:
pk = db.db_engine.execute(stmt).first()[0]

In [5]:
call = API_call(pk)

In [13]:
#call.game_batting_stats

In [7]:
records = call.game_player_links

In [8]:
new_list = []
for record in records:
    new_dict = {}
    for k,v in record.items():
        if k.startswith('stats_batting_'):
            new_dict[k.replace('stats_batting_','')]=v
    new_list.append(new_dict)

In [19]:
#item for sublist in list for item in sublist 
r=[ 
    { 
        k.replace('stats_batting_','') : v 
        if k.startswith('stats_batting_')
        else k:v
        for k,v in record.items() 
        
    } 
    
    for record in records
]


SyntaxError: invalid syntax (<ipython-input-19-018501a1c3f6>, line 6)

In [ ]:
# item for sublist in list for item in sublist

game_stat= list(
    { k for keys in call.game_player_links 
     for k in keys if k.startswith('stats_batting') }
)

In [ ]:
season_stat = list(
    { k for keys in call.game_player_links 
     for k in keys if k.startswith('seasonStats_pitching') }
)

In [ ]:
game_stat

In [ ]:
season_stat

In [ ]:
df = pd.DataFrame.from_records(call.game_player_links)[game_stat]

In [ ]:
df[df['stats_batting_stolenBasePercentage']==".---"][['stats_batting_stolenBases','stats_batting_caughtStealing']]


In [ ]:
df[df['stats_batting_stolenBasePercentage'].isna()==True]


In [ ]:
pk = gamePks['2018'][606]

ins = db.insert_game(pk,replace=True)

pd.DataFrame.from_records(ins)

In [ ]:
for table in tables:
    db.db_engine.execute(f'delete from {table} where gamePk={pk}')

In [ ]:
for table in tables:
    r = db.db_engine.execute(f"select count(*) from {table} where gamePk={pk}")
    print(r.fetchone()[0])

In [ ]:
r.fetchone()[0]

In [ ]:
pks = gamePks['2019'][500:550]

calls = [API_call(pk) for pk in pks]
#item for sublist in list for item in sublist
records = [link for call in calls for link in call.game_player_links]

In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
df['stats_pitching_gamesPitched'].value_counts()

In [ ]:
df['gameStatus_isCurrentBatter'].value_counts()

In [ ]:
# #TEST: which columns, if any, don't have any values
# dfs = {}
# for table in db.db_engine.table_names():
#     df = pd.read_sql(f"select * from {table}",db.db_engine)
#     dfs[table]=df


# for k in dfs.keys():
#     df = dfs[k]
#     for col in df.columns:
#         if df[col].isna().sum()==df.shape[0]:
#             print(k,col)

# pks = [x[0] for x in session.query(Game.pk).all()]

# calls = [API_call(pk) for pk in pks]

# dfs_fromApi = {}
# for k in calls[0].__dict__.keys():
#     if k[0] != '_':
#         records = [x for call in calls for x in call.__dict__[k]]
#         df = pd.DataFrame.from_records(records)
#         dfs_fromApi[k] = df

In [ ]:
call = API_call(567055)

In [ ]:
liveData = call._result['liveData']

In [ ]:
box_score = liveData['boxscore']

In [ ]:
away = box_score['teams']['away']

In [ ]:
players = away['players']

In [ ]:
player_dicts = []
for k in players.keys():
    player_dicts.append(flatten_dicts(players[k]))



In [ ]:
df = pd.DataFrame.from_records(player_dicts)

In [ ]:
df.info()

In [ ]:
df['status_description'].value_counts()

In [ ]:
df.info(verbose=True)

In [ ]:
stmt = """
select 
    pk,
    count(pk) as count
from
    games
group by 
    pk
having 
    count > 1
"""

In [ ]:
db.db_engine.execute(stmt).fetchall()